# Head

In [1]:
from google.colab import drive
import os
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/Othercomputers/retep's PC/MambaVAE")

Mounted at /content/gdrive


In [ ]:
%pip install mambapy datasets

In [ ]:
import torch, transformers, importlib
from transformers import MambaConfig, MambaForCausalLM, AutoTokenizer
import InitMamba, VAE, Dataset

# Lab

In [ ]:
!python Trainer.py